## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data</a>
2. <a href="#item42">Part 1 - Building and Training Models</a>
3. <a href="#item43">Part 2 - Evaluating the Models</a>  
4. <a href="#item44">Part 3 - Prediction using the Models</a>  

</font>
    
</div>

## Download Data

In [ ]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [ ]:
!unzip concrete_data_week4.zip

## Part 1 - Building and Training Models

Here, the VGG16 and Resnet50 pre-trained Convolutional Neural Networks will be used, with its topmost layer replaced with our own dense layer of 2 neurons (either cracked wall or uncracked wall).

In [1]:
# Import required packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input as preprocess_input_rs50
from keras.models import load_model

In [2]:
# Define the Global Constants
image_size = 224
batch_size = 100

In [3]:
# Create ImageDataGenerator with VGG16 preprocessing
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

train_generator = data_generator.flow_from_directory(
    'concrete_data/train',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
    'concrete_data/valid',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [4]:
# Create the model
model = Sequential()

# Set the pooling and weights to be the same as Resnet50 for fair comparison
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

# Add the Dense Output Layer with 2 neurons and a softmax activation
model.add(Dense(2, activation='softmax'))

# Print the layers in the model
model.layers

In [5]:
# Set the VGG16 pretrained model parameters to untrainable (it is already pre-trained)
model.layers[0].trainable = False

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [6]:
# Compile the model using the adam optimizer and a loss function of categorical cross-entropy
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# Define the same epoch and steps per epoch as the Resnet50 model
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [8]:
# Fit the model
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/2
301/301 [==============================] - 4500s 15s/step - loss: 0.1214 - accuracy: 0.9616 - val_loss: 0.0310 - val_accuracy: 0.9939
Epoch 2/2
301/301 [==============================] - 4538s 15s/step - loss: 0.0244 - accuracy: 0.9947 - val_loss: 0.0188 - val_accuracy: 0.9964


In [9]:
# Save the model
model.save('classifier_vgg16_model.h5')

In [10]:
# Create ImageDataGenerator with Resnet50 preprocessing
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input_rs50,
)

train_generator = data_generator.flow_from_directory(
    'concrete_data/train',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
    'concrete_data/valid',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical')

# Now create the Resnet50 model to compare with 
model_rs50 = Sequential()

model_rs50.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

# Add the Dense Output Layer with 2 neurons
model_rs50.add(Dense(2, activation='softmax'))

# Print the layers in the model
model_rs50.layers

# Set the Resnet50 pretrained model parameters to untrainable
model_rs50.layers[0].trainable = False

# Print the model summary
model_rs50.summary()

# Compile the model using the adam optimizer
model_rs50.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
fit_history = model_rs50.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

# Save the model
model_rs50.save('classifier_rs50_model.h5')

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________
Epoch 1/2
301/301 [==============================] - 2007s 7s/step - loss: 0.0490 - accuracy: 0.9829 - val_loss: 0.0095 - val_accuracy: 0.9977
Epoch 2/2
301/301 [==============================] - 2051s 7s/step - loss: 0.0072 - accuracy: 0.9983 - val_loss: 0.0062 - val_accuracy: 0.9985


<a id="item43"></a>

## Part 2 - Evaluating the Models

Here, the two different CNN models will be evaluated and its performance compared.


In [11]:
# Load the VGG16 model
model_vgg16 = load_model('classifier_vgg16_model.h5')
model_vgg16.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [12]:
# Create ImageDataGenerator with VGG16 preprocessing
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

test_generator_vgg16 = data_generator.flow_from_directory(
    'concrete_data/test',
    target_size=(image_size, image_size),
    shuffle = False)

Found 500 images belonging to 2 classes.


In [13]:
# Evaluate the model on test data
score = model_vgg16.evaluate_generator(test_generator_vgg16)
accuracy_vgg16 = score[1]
print("VGG16 Accuracy: ", accuracy_vgg16)

Instructions for updating:
Please use Model.evaluate, which supports generators.
VGG16 Accuracy:  0.9959999918937683


In [14]:
# Load the Resnet50 Model
model_rs50 = load_model('classifier_rs50_model.h5')
model_rs50.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [15]:
# Create ImageDataGenerator with Resnet50 preprocessing
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input_rs50,
)

test_generator_rs50 = data_generator.flow_from_directory(
    'concrete_data/test',
    target_size=(image_size, image_size),
    shuffle = False)

Found 500 images belonging to 2 classes.


In [16]:
# Evaluate the model on test data
score = model_rs50.evaluate_generator(test_generator_rs50)
accuracy_rs50 = score[1]
print("ResNet50 Accuracy: ", accuracy_rs50)

ResNet50 Accuracy:  1.0


In [ ]:
# As can be seen, the ResNet50 Accuracy is 1.0 compared to VGG16 Accuracy of 0.996, indicating the ResNet50 performs marginally better

## Part 3 - Prediction using the Models

This section will perform simple prediction of five input images. 'Positive' indicates the image contains a cracked wall, whilst 'Negative' indicates an uncracked wall.

In [23]:
# Using the VGG16 Model
predict_vgg16 = model_vgg16.predict_generator(test_generator_vgg16)
prob_positive_vgg16 = predict_vgg16[0:5][:,0]
prediction_vgg16 = ['Positive' if x > 0.5 else 'Negative' for x in prob_positive_vgg16]
prediction_vgg16

['Positive', 'Positive', 'Positive', 'Positive', 'Positive']

In [24]:
# Using the ResNet50 Model
predict_rs50 = model_rs50.predict_generator(test_generator_rs50)
prob_positive_rs50 = predict_rs50[0:5][:,0]
prediction_rs50 = ['Positive' if x > 0.5 else 'Negative' for x in prob_positive_rs50]
prediction_rs50

['Positive', 'Positive', 'Positive', 'Positive', 'Positive']

In [ ]:
# Both models made the same prediction for the 5 input images